# lec08
## Simple CNN

In [12]:
from torch import nn, optim, cuda
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import time

In [25]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1) # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)

## Advanced CNN
- Inception Module

In [84]:
class InceptionA(nn.Module):

    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

In [87]:
class AdvancedCNN(nn.Module):

    def __init__(self):
        super(AdvancedCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return F.log_softmax(x)

## MNIST Dataset
### Simple CNN

In [6]:
batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')

train_dataset = datasets.MNIST(root='./mnist_data/',
                               train = True,
                               transform=transforms.ToTensor(),
                               download=True)
test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=False)

Training MNIST Model on cuda


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [26]:
model = CNN()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [82]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {0} | Batch Status: {1}/{2} ({3:.0f}%) | Loss: {4:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    test_loss /= len(test_loader.dataset)
    print('{0}\nTest set: Average loss: {1:.4f}, Accuracy: {2} / {3} ({4:.0f}%)'.format(
        '=' * 50, test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

In [33]:
start = time.time()
for epoch in range(1, 10):
    epoch_start = time.time()
    train(epoch)
    m, s = divmod(time.time() - epoch_start, 60)
    print('Training time: {0:.0f}m {1:.0f}s'.format(m, s))
    test()
    m, s = divmod(time.time() - epoch_start, 60)
    print('Testing time: {0:.0f}m {1:.0f}s'.format(m, s))
m, s = divmod(time.time() - start, 60)
print('Total Time: {0:.0f}m {1:.0f}s\nModel was trained on {2}'.format(m, s, device))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 0.023902
Train Epoch: 1 | Batch Status: 640/60000 (1%) | Loss: 0.076838
Train Epoch: 1 | Batch Status: 1280/60000 (2%) | Loss: 0.055935
Train Epoch: 1 | Batch Status: 1920/60000 (3%) | Loss: 0.035297
Train Epoch: 1 | Batch Status: 2560/60000 (4%) | Loss: 0.096614
Train Epoch: 1 | Batch Status: 3200/60000 (5%) | Loss: 0.095446
Train Epoch: 1 | Batch Status: 3840/60000 (6%) | Loss: 0.048012
Train Epoch: 1 | Batch Status: 4480/60000 (7%) | Loss: 0.102199
Train Epoch: 1 | Batch Status: 5120/60000 (9%) | Loss: 0.051036
Train Epoch: 1 | Batch Status: 5760/60000 (10%) | Loss: 0.014268
Train Epoch: 1 | Batch Status: 6400/60000 (11%) | Loss: 0.125097
Train Epoch: 1 | Batch Status: 7040/60000 (12%) | Loss: 0.052828
Train Epoch: 1 | Batch Status: 7680/60000 (13%) | Loss: 0.092560
Train Epoch: 1 | Batch Status: 8320/60000 (14%) | Loss: 0.025558
Train Epoch: 1 | Batch Status: 8960/60000 (15%) | Loss: 0.083771
Train Epoch: 1 | Batch Status: 9600/60

### Advanced CNN

In [88]:
model = AdvancedCNN()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [89]:
start = time.time()
for epoch in range(1, 10):
    epoch_start = time.time()
    train(epoch)
    m, s = divmod(time.time() - epoch_start, 60)
    print('Training time: {0:.0f}m {1:.0f}s'.format(m, s))
    test()
    m, s = divmod(time.time() - epoch_start, 60)
    print('Testing time: {0:.0f}m {1:.0f}s'.format(m, s))
m, s = divmod(time.time() - start, 60)
print('Total Time: {0:.0f}m {1:.0f}s\nModel was trained on {2}'.format(m, s, device))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Train Epoch: 9 | Batch Status: 10060/60000 (17%) | Loss: nan
Train Epoch: 9 | Batch Status: 10070/60000 (17%) | Loss: nan
Train Epoch: 9 | Batch Status: 10080/60000 (17%) | Loss: nan
Train Epoch: 9 | Batch Status: 10090/60000 (17%) | Loss: nan
Train Epoch: 9 | Batch Status: 10100/60000 (17%) | Loss: nan
Train Epoch: 9 | Batch Status: 10110/60000 (17%) | Loss: nan
Train Epoch: 9 | Batch Status: 10120/60000 (17%) | Loss: nan
Train Epoch: 9 | Batch Status: 10130/60000 (17%) | Loss: nan
Train Epoch: 9 | Batch Status: 10140/60000 (17%) | Loss: nan
Train Epoch: 9 | Batch Status: 10150/60000 (17%) | Loss: nan
Train Epoch: 9 | Batch Status: 10160/60000 (17%) | Loss: nan
Train Epoch: 9 | Batch Status: 10170/60000 (17%) | Loss: nan
Train Epoch: 9 | Batch Status: 10180/60000 (17%) | Loss: nan
Train Epoch: 9 | Batch Status: 10190/60000 (17%) | Loss: nan
Train Epoch: 9 | Batch Status: 10200/60000 (17%) | Loss: nan
Train Epoch: 9 | Batch Status: 10210/60000 (17%) 